### 公众号：数据不吹牛，更多案例和有趣分析等你来撩

In [23]:
import pandas as pd
import re
import numpy as np
import os

In [6]:
os.chdir('C:\\Users\\Administrator\\教程\\interest analyse')

In [24]:
df = pd.read_excel('李子柒辣椒酱评论.xlsx')
print('一共 %d 条评论数据' % len(df))
df.head()

一共 1794 条评论数据


,买家,初评内容
0,蓝**e,诚信的卖家，暴力快递碎了一瓶马上补发了，吃完继续，祝老板生意兴隆！
1,自**1,客服态度非常好（我主要是看重售后服务）、物流也棒棒的、李子柒酱超级好吃😋、有了李子柒牛肉酱、...
2,浓**浆,爱了，超符合口味，太喜欢了 李子柒我爱你
3,刘**芸,味道很不错！
4,林**3,好吃，不辣，稍贵


### 把评价分成短句

In [25]:
result = []
for i in df['初评内容']:
    #根据标点符号来拆分内容
    result.append(re.split('[:：,，.。!！~·`\;； ……、]',i))
result[:5]

[['诚信的卖家', '暴力快递碎了一瓶马上补发了', '吃完继续', '祝老板生意兴隆', ''],
 ['客服态度非常好（我主要是看重售后服务）',
  '物流也棒棒的',
  '李子柒酱超级好吃😋',
  '有了李子柒牛肉酱',
  '麻麻再也不用担心我吃不饱啦',
  ''],
 ['爱了', '超符合口味', '太喜欢了', '李子柒我爱你'],
 ['味道很不错', ''],
 ['好吃', '不辣', '稍贵']]

### 以物流为例，做个判断

In [26]:
for word in result[0]:
    #先判断是不是物流相关的
    if '物流' in word or '快递' in word or '配送' in word or '取货' in word:
        #再判断是正面还是负面情感
        if '好' in word or '不错' in word or '棒' in word or '满意' in word or '迅速' in word:
            print('物流正面倾向')
        elif '慢' in word or '龟速' in word or '暴力' in word or '差' in word:
            print('物流负面倾向')

物流负面倾向


### 实现代码

In [28]:
result = []
    
for i in df['初评内容']:
    #根据标点符号来拆分内容
    result.append(re.split('[:：,，.。!！~·`\;； ……、]',i))
        
def judge_comment(df,result):
    
    judges = pd.DataFrame(np.zeros(13 * len(df)).reshape(len(df),13),
                      columns = ['品牌','物流正面','物流负面','包装正面','包装负面','原料正面',
                                 '原料负面','口感正面','口感负面','日期正面','日期负面',
                                 '性价比正面','性价比负面'])
    
    for i in range(len(result)):
        
        words = result[i]
        for word in words:
            #李子柒的产品具有强IP属性，基本都是正面评价，这里不统计情绪，只统计提及次数
            if '李子柒' in word or '子柒' in word or '小柒' in word or '李子七' in word\
                or '小七' in word:
                    judges.iloc[i]['品牌'] = 1

            #先判断是不是物流相关的
            if '物流' in word or '快递' in word or '配送' in word or '取货' in word:
                #再判断是正面还是负面情感
                if '好' in word or '不错' in word or '棒' in word or '满意' in word or '迅速' in word:
                    judges.iloc[i]['物流正面'] = 1
                elif '慢' in word or '龟速' in word or '暴力' in word or '差' in word:
                    judges.iloc[i]['物流负面'] = 1

            #判断是否包装相关
            if '包装' in word or '盒子' in word or '袋子' in word or '外观' in word:
                if '高端' in word or '大气' in word or '还行' in word or '完整' in word or '好' in word or\
                   '严实' in word or '紧' in word:
                    judges.iloc[i]['包装正面'] = 1
                elif  '破' in word or '破损' in word or '瘪' in word or '简陋' in word:
                    judges.iloc[i]['包装负面'] = 1

            #产品
            #产品原料是牛肉为主，且评价大多会提到牛肉，因此我们把这个单独拎出来分析
            if '肉' in word:
                if '大' in word or '多' in word or '足' in word or '香' in word or '才' in word:
                    judges.iloc[i]['原料正面'] = 1
                elif '小' in word or '少' in word or '没' in word:
                    judges.iloc[i]['原料负面'] = 1

            #口感的情绪
            if '口味' in word or '味道' in word or '口感' in word or '吃起来' in word:
                if '不错' in word or '好' in word or '棒' in word or '鲜' in word or\
                    '可以' in word or '喜欢' in word or '符合' in word:
                    judges.iloc[i]['口感正面'] = 1
                elif '不好' in word or '不行' in word or '不鲜' in word or\
                    '太烂' in word:
                    judges.iloc[i]['口感负面'] = 1

            #口感方面，有些是不需要出现前置词，消费者直接评价好吃难吃的，例如：
            if '难吃' in word or '不好吃' in word:
                judges.iloc[i]['口感负面'] = 1
            elif '好吃' in word or '香' in word:
                judges.iloc[i]['口感正面'] = 1

            #日期是不是新鲜
            if '日期' in word or '时间' in word or '保质期' in word:
                if '新鲜' in word:
                    judges.iloc[i]['日期正面'] = 1
                elif '久' in word or '长' in word:
                    judges.iloc[i]['日期负面'] = 1
            elif '过期' in word:
                judges.iloc[i]['日期负面'] = 1

            #性价比
            if '划算' in word or '便宜' in word or '赚了' in word or '囤货' in word or '超值' in word or \
                '太值' in word or '物美价廉' in word or '实惠' in word or '性价比高' in word or '不贵' in word: 
                judges.iloc[i]['性价比正面'] = 1
            elif  '贵' in word or '不值' in word or '亏了' in word or '不划算' in word or '不便宜' in word:
                judges.iloc[i]['性价比负面'] = 1
                
    final_result = pd.concat([df,judges],axis = 1)
    
    return final_result

### 计算之

In [11]:
judges = judge_comment(df,result)

In [27]:
judges.describe()
judges.head(3)

,买家,初评内容,品牌,物流正面,物流负面,包装正面,包装负面,原料正面,原料负面,口感正面,口感负面,日期正面,日期负面,性价比正面,性价比负面
0,蓝**e,诚信的卖家，暴力快递碎了一瓶马上补发了，吃完继续，祝老板生意兴隆！,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,自**1,客服态度非常好（我主要是看重售后服务）、物流也棒棒的、李子柒酱超级好吃😋、有了李子柒牛肉酱、...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,浓**浆,爱了，超符合口味，太喜欢了 李子柒我爱你,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### 次数和占比计算

In [30]:
rank = judges.iloc[:,2:].sum().reset_index().sort_values(0,ascending = False)
rank.columns = ['分类','提及次数']
rank['占比'] = rank['提及次数'] / rank['提及次数'].sum()
rank['高级分类'] = rank['分类'].str[:-2]
rank

,分类,提及次数,占比,高级分类
7,口感正面,1107.0,0.570619,口感
5,原料正面,324.0,0.167010,原料
0,品牌,186.0,0.095876,
11,性价比正面,95.0,0.048969,性价比
3,包装正面,68.0,0.035052,包装
12,性价比负面,58.0,0.029897,性价比
6,原料负面,53.0,0.027320,原料
8,口感负面,19.0,0.009794,口感
1,物流正面,10.0,0.005155,物流
2,物流负面,9.0,0.004639,物流


### 原料负面

In [32]:
source = judges.loc[judges['原料负面'] == 1,]
source_sum = source.sum().reset_index().loc[2:,:].sort_values(0,ascending = False)
source_sum.columns = ['分类','提及次数']
source_sum.loc[source_sum['提及次数'] > 0,:]

,分类,提及次数
8,原料负面,53
9,口感正面,24
7,原料正面,8
2,品牌,3
5,包装正面,2
13,性价比正面,2
14,性价比负面,2


In [31]:
source.head()

,买家,初评内容,品牌,物流正面,物流负面,包装正面,包装负面,原料正面,原料负面,口感正面,口感负面,日期正面,日期负面,性价比正面,性价比负面
24,绿**辰,好吃，只是牛肉太少了,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
56,香**蓉,牛肉酱，味道不错。牛肉比较少，有时候一瓶有很多肥的，很无语的感觉。,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
74,开**洲,不吹不黑，事实说话，味道还可以，不过说好的牛肉只是很小很小很一小块，牛肉块数量也屈指可数，这...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
86,凝**贝,肉粒不小，老公喜欢吃,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
92,冥**天,还好吧，没啥牛肉味,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### 性价比负面

In [33]:
price = judges.loc[judges['性价比负面'] == 1,]
price_sum = price.sum().reset_index().loc[2:,:].sort_values(0,ascending = False)
price_sum.columns = ['分类','提及次数']
price_sum.loc[price_sum['提及次数'] > 0,:]

,分类,提及次数
14,性价比负面,58
9,口感正面,44
7,原料正面,8
13,性价比正面,7
2,品牌,3
8,原料负面,2
4,物流负面,1
5,包装正面,1


In [36]:
price.head()

,买家,初评内容,品牌,物流正面,物流负面,包装正面,包装负面,原料正面,原料负面,口感正面,口感负面,日期正面,日期负面,性价比正面,性价比负面
4,林**3,好吃，不辣，稍贵,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
13,穆**麦,第一次吃觉得一般，后来吃了其他的牛肉酱之后，发现这个真的非常好吃，但平常价格也是真的有点贵啊...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
31,霹**袁,味道挺不错，可是太贵了 本来想着59三瓶还行 结果两瓶 不太划算,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
41,吕**三,很好吃，就是价格太贵了,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
44,莎**4,感觉还是在直播间买的更划算一点，要是平常买的话太贵了。,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


### 物流负面

In [35]:
judges.loc[judges['物流负面'] == 1,].iloc[:,:2]

,买家,初评内容
0,蓝**e,诚信的卖家，暴力快递碎了一瓶马上补发了，吃完继续，祝老板生意兴隆！
314,原**像,一般，除了贵，物流也不是一般差，极差！！！！
315,辽**李,物流太慢了
319,恋**你,双十一买的，价格很划算，还没吃，不知道味道怎么样，物流挺慢的，很久才收到
450,全**7,物流极慢！，二十多天才收到货，无语了！极差，爱理不理
632,妖**吖,快递实在是太慢啦
790,厌**8,双十一最慢的一个快递
1693,胡**5,超级好评，味道服务都是非常好的。一共寄了三次来，前面两次因为快递太暴力了，各种漏油，瓶子都摔...
1700,一**郝,牛肉粒很大，不错，味道也很好，货真价实的好，就是邮政快递感觉有点慢，等不及的想品尝这个酱，总...
